In [49]:
# Desconexión de la plataforma MetaTrader 5
mt5.shutdown()
globalMaxGross = 0
globalPositionType = ""
globalGross = 0
globalPrice = 0
globalPositionOpen = 0
globalPositionStop = 0
globalPositionTrailingN = 0
globalPositionTrailingV = 0

In [50]:
import MetaTrader5 as mt5
import time

In [51]:
# Conexión con la plataforma MetaTrader 5
if not mt5.initialize():
    print("Error al inicializar MetaTrader 5.")
    quit()

In [52]:
#configuration
globalSpreadBuy = 2
globalSpreadSell = 2
globalTrailing = 2
globalTrailingStop = 1
globalSymbol = "EURUSD"
globalVolume = 0.60

globalLastPrice = 0
globalPositionOpen = 0
globalPositionStop = 0
globalPositionType = ""
globalGross = 0
globalPrice = 0
globalMaxGross = 0
globalPositionTrailingN = 0
globalPositionTrailingV = 0

In [53]:
positions = mt5.positions_get()
if positions:
    position = mt5.positions_get()[0]
    print(position.ticket)
else:
    print("No positions found")

No positions found


In [54]:
def priceConvert(price):
    price = format(price, '.5f')
    if len(price) == 5:
        price += '0'
    return int(str(str(price).replace(".", ""))[:6])

In [55]:
def priceConvertDot(price):
    return str(price)[:1] + "." + str(price)[1:]

In [56]:
def closePosition(pos):
    close_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": pos.symbol,
        "volume": globalVolume,
        "type": mt5.ORDER_TYPE_BUY if pos.type == 1 else mt5.ORDER_TYPE_SELL,
        "position": pos.ticket,
        "price": mt5.symbol_info_tick(pos.symbol).bid,
        "deviation": 20,
        "magic": 234000,
        "comment": 'Close trade',
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    # Envío de la orden de cierre al servidor de MetaTrader 5
    result = mt5.order_send(close_request)

    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("Error al cerrar la posición: ", result.comment)
    else:
        print("Posición cerrada con éxito. Resultado: ", result)
        global globalMaxGross
        global globalSpreadBuy
        global globalSpreadSell
        global globalPositionType
        global globalGross
        global globalPrice
        global globalPositionOpen
        global globalPositionStop
        global globalPositionTrailingN
        global globalPositionTrailingV
        globalMaxGross = 0
        globalPositionType = ""
        globalGross = 0
        globalPrice = 0
        globalPositionOpen = 0
        globalPositionStop = 0
        globalPositionTrailingN = 0
        globalPositionTrailingV = 0

In [57]:
def printInfo(type):
    if type == 1:
        print("-------------------")
        print("Tipo: "+str(globalPositionType))               
        print("Ganacia: "+str(globalGross))
        print("Open: "+str(globalPositionOpen))
        print("Precio: "+str(globalPrice))
        print("TrailingN: "+str(globalPositionTrailingN))
        print("TrailingV: "+str(globalPositionTrailingV))
        print("Stop: "+str(globalPositionStop))
        print("Diff: "+str(globalPositionTrailingV - globalPositionTrailingN))
        print("-------------------")

In [58]:
def getActivePosition(price):
    global globalMaxGross
    global globalSpreadBuy
    global globalSpreadSell
    global globalPositionType
    global globalGross
    global globalPrice
    global globalPositionOpen
    global globalPositionStop
    global globalPositionTrailingN
    global globalPositionTrailingV
    global globalTrailingStop
        
    globalPrice = priceConvert(price)
    
    trades = mt5.positions_get() 
    if trades:   
        pos = mt5.positions_get()[0] 
        globalGross = pos.profit
        globalPositionOpen = priceConvert(pos.price_open)           
        
        if pos.type == 1:
            globalPositionType = "Sell"
            globalPositionTrailingN = globalPositionTrailingN if globalPositionTrailingN > 0 else globalPositionOpen - globalSpreadSell
            globalPositionTrailingV = globalPositionTrailingV if globalPositionTrailingV > 0 else globalPositionTrailingN
                   
            if globalPrice < globalPositionTrailingN:
                globalPositionTrailingN = globalPrice
                
                if (globalPositionTrailingV - globalPositionTrailingN) >= globalTrailing: 
                    globalPositionStop = globalPositionTrailingN + globalTrailingStop
                    globalPositionTrailingV = globalPositionTrailingN
        
            if globalPositionStop > 0:
                if globalPrice >= globalPositionStop and globalGross > 0:
                    print("Closed at: " + str(globalGross))
                    closePosition(pos)
        else:
            globalPositionType = "Buy"
            globalPositionTrailingN = globalPositionTrailingN if globalPositionTrailingN > 0 else globalPositionOpen + globalSpreadSell
            globalPositionTrailingV = globalPositionTrailingV if globalPositionTrailingV > 0 else globalPositionTrailingN
            
            if globalPrice > globalPositionTrailingN:
                globalPositionTrailingN = globalPrice
                if (globalPositionTrailingN - globalPositionTrailingV) >= globalTrailing: 
                    globalPositionStop = globalPositionTrailingN - globalTrailingStop
                    globalPositionTrailingV = globalPositionTrailingN
                    print("New TrailingStop: "+ str(globalPositionStop))
                    
            if globalPositionStop > 0:
                if globalPrice <= globalPositionStop and globalGross > 0:
                    print("Closed at: " + str(globalGross))
                    closePosition(pos)  
        
        #printInfo(1)
        
    else:     
        globalMaxGross = 0
        globalGross = 0
        globalPositionOpen = 0
        globalPositionStop = 0
        globalPositionTrailingN = 0
        globalPositionTrailingV = 0

In [ ]:
while True:
    tick = mt5.symbol_info_tick(globalSymbol)
    if tick is not None:

        if tick.bid != globalLastPrice:
            globalLastPrice = tick.bid
            #price = tick.bid 
            getActivePosition(tick.bid)
            #print(priceConvert(tick.bid))
    #time.sleep(1)

Closed at: 1.8
Posición cerrada con éxito. Resultado:  OrderSendResult(retcode=10009, deal=11820372, order=14333265, volume=0.6, price=1.0765500000000001, bid=0.0, ask=0.0, comment='Request executed', request_id=2442108186, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='EURUSD', volume=0.6, price=1.0765500000000001, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=1, type_time=0, expiration=0, comment='Close trade', position=14333109, position_by=0))


In [ ]:
# Creación de una orden de mercado
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": volume,
    "type": mt5.ORDER_TYPE_SELL,
    "price": mt5.symbol_info_tick(symbol).ask,
    "deviation": 20,
    "magic": 234000,
    "comment": "Python Script Order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}

# Envío de la orden al servidor de MetaTrader 5
result = mt5.order_send(request)

if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("Error al enviar la orden: ", result.comment)
else:
    print("La orden ha sido enviada con éxito. Resultado: ", result)



In [ ]:
positions = mt5.positions_get()
position_ticket = 0
deal = result.deal
# imprimir información de las posiciones abiertas
if positions:
    for position in positions:
        position_ticket = position.ticket
        print(position.ticket)
        print(position.symbol)
        print(deal)
else:
    print("No positions found")

print(mt5.TRADE_ACTION_DEAL)
print(mt5.ORDER_TYPE_BUY)
print(mt5.ORDER_TIME_GTC)
print(mt5.ORDER_FILLING_IOC)

In [ ]:
# Cierre de la posición

if position_id:
    close_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_TYPE_BUY,
        "position": position_ticket,
        "price": mt5.symbol_info_tick(symbol).bid,
        "deviation": 20,
        "magic": 234000,
        "comment": 'Close trade',
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC, # also tried with ORDER_FILLING_RETURN
    }

    # Envío de la orden de cierre al servidor de MetaTrader 5
    result = mt5.order_send(close_request)

    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print("Error al cerrar la posición: ", result.comment)
    else:
        print("Posición cerrada con éxito. Resultado: ", result)

